<a href="https://colab.research.google.com/github/PeterHJY628/tutorial_notebooks/blob/main/CIFAR_10H_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloning repository of CIFAR-10H Annotation
Paper: Human uncertainty makes classification more robust (https://arxiv.org/pdf/1908.07086.pdf)

Label:  CIFAR10 [0: airplane, 1: automobile, 2: bird, 3: cat, 4: deer, 5: dog, 6: frog, 7: horse, 8: ship, 9: truck] <br>

<img src="https://miro.medium.com/max/1010/1*r8S5tF_6naagKOnlIcGXoQ.png" alt="alternatetext">




In [ ]:
!git clone https://github.com/jcpeterson/cifar-10h
%cd cifar-10h

Cloning into 'cifar-10h'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 48 (from 1)
Receiving objects: 100% (49/49), 10.85 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/cifar-10h


# main script

In [ ]:
import torch
import torch.nn as nn #provide layers AF LF
import torch.optim as optim #optimizer
import torch.nn.functional as F #provide function version of operation
import torchvision #package for image data, db,process...
from torchvision import models #pretrained and predefined model
import torchvision.transforms as transforms #image preprocess and Data Augmentation
import os #Interact with OS
import argparse # help to parse command line argument and
import copy #shallow copy: both change; deep copy: change one , original do not change
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)#分别为 PyTorch 的 CPU 和 GPU 操作设置随机种子。
    np.random.seed(seed)#设置 NumPy 的随机数生成器的种子
    os.environ['PYTHONHASHSEED'] = str(seed)#通过设置环境变量 PYTHONHASHSEED 来控制 Python 的哈希种子。这样可以确保在哈希操作中（如字典或集合的键）生成的结果是一致的，这在使用随机数据或模型时是很重要的。 确保 Python 的基于哈希的操作是确定性的
    torch.backends.cudnn.deterministic = True #keep deterministic  保证 CUDA 操作的确定性。
    torch.backends.cudnn.benchmark = False #no benchmark 禁用基准模式，使操作是确定性的，但可能会稍微降低运行速度。
#help to adjust parameters like learning rate, batch size and so on
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=1024, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=2048, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=10, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()# change model to train mode
    for batch_idx, (inputs, targets, ad) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)#forward
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()#update parameters of the model according to former calculated gradient

def test(model, testloader):
    model.eval()# change model to test mode
    correct = 0#用于记录模型在测试集中预测正确的样本数。
    total = 0#用于记录测试集中样本的总数。
    with torch.no_grad():# no gradient calculation since no backward needed
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)#find the maximum value in the 1st dimension
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()#total += targets.size(0)：将当前批次的样本数累加到总样本数中。
#predicted.eq(targets)：比较预测值与真实值，返回一个布尔张量，表示哪些样本的预测是正确的。
#.sum().item()：将布尔张量中的 True 值（即正确预测的样本数）求和，并转换为 Python 的标量。
#correct += ...：将本批次正确预测的数量累加到 correct 变量中。
    return correct / total#return accuracy rate

# CIFAR-10H dataloader

In [ ]:
from PIL import Image# Python Imaging Library（PIL）中的 Image 模块的语句。PIL 是一个用于打开、操作和保存多种图像文件格式的库，而 Pillow 是其更现代的分支和替代品，目前广泛使用。使用这条导入语句时，我们实际上是在导入 Pillow 提供的 Image 模块。
import numpy as np
import torchvision

class CIFAR10H(torchvision.datasets.CIFAR10):#Inherit from torchvision.datasets.CIFAR10 class

    def __init__(self, root,  rand_number=0, train=False, transform=None, target_transform=None,
                 download=False):
        super(CIFAR10H, self).__init__(root, train, transform, target_transform, download)
        self.transform = transform
        self.target_transform = target_transform
        self.ad = np.load(os.path.join(root,'cifar10h-probs.npy'))#additional data
    """
    self 允许引用当前实例
    root：数据集的根目录。
rand_number：一个随机数参数，这里没有使用。
train：是否加载训练集。True 加载训练集，False 加载测试集。
transform：应用于图像的转换（如数据增强、归一化等）。
target_transform：应用于标签的转换。
download：如果数据集不存在，是否自动下载。
super(CIFAR10H, self).__init__(...) 调用父类的初始化方法，初始化标准 CIFAR-10 数据集。
self.ad = np.load(os.path.join(root, 'cifar10h-probs.npy')) 加载一个名为 cifar10h-probs.npy 的文件，文件路径是 root 目录下。这通常是一个包含额外注释信息的 .npy 文件（NumPy 格式），例如 CIFAR-10H 数据集中的概率标签。
    """

    def __getitem__(self, index: int):#用于获取数据集中的单个样本（图像、标签和附加数据
        img, target = self.data[index], self.targets[index]#self.data 是包含所有图像的 NumPy 数组。self.targets 是包含所有图像标签的数组
        img = Image.fromarray(img)#将 NumPy 数组格式的图像转换为 PIL 图像对象。这是因为许多图像处理操作在 PIL 图像对象上更方便。
        ad = self.ad[index]#从 self.ad 中获取与当前索引对应的附加数据（如概率值或其他信息）。self.ad 是在类初始化时加载的 NumPy 数组，包含了与每个图像相关的附加信息。
        if self.transform is not None:
            img = self.transform(img)#应用图像转换：如果指定了图像转换（self.transform），则对图像应用这些转换（如数据增强、归一化等）。这通常用于训练过程中，以提高模型的泛化能力。
        if self.target_transform is not None:
            target = self.target_transform(target)#如果指定了标签转换（self.target_transform），则对目标标签应用这些转换。这可能用于将标签映射到其他形式，或者进行其他预处理操作。
        return img, target, ad#方法返回一个包含图像、目标标签和附加数据的三元组。调用此方法时，可以获取到对应索引的完整样本信息。

# Run script

In [ ]:
seed_everything()
mean_cifar10, std_cifar10 = (0.5071, 0.4866, 0.4409), (0.2009, 0.1984, 0.2023)#均值与标准差：这些值是 CIFAR-10 数据集中每个通道（RGB）的均值和标准差，通常用于图像的标准化处理。mean_cifar10 是每个颜色通道的均值。std_cifar10 是每个颜色通道的标准差。
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(), transforms.ToTensor(),
            transforms.Normalize(mean_cifar10, std_cifar10), ])
'''训练集的转换
transforms.Compose([...])：将多个图像变换组合在一起。
transforms.RandomCrop(32, padding=4)：随机裁剪图像为 32x32 像素，同时在边缘填充 4 个像素。这有助于增加图像的多样性。
transforms.RandomHorizontalFlip()：以 50% 的概率随机水平翻转图像。这是另一种数据增强方式，用于提高模型的泛化能力。
transforms.ToTensor()：将图像从 PIL 格式转换为 PyTorch 张量，并将像素值缩放到 [0, 1] 范围内。
transforms.Normalize(mean_cifar10, std_cifar10)：使用之前定义的均值和标准差对图像进行标准化处理，使其均值为 0，标准差为 1，这样可以加速训练和提高模型的性能。'''

transform_test = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize(mean_cifar10, std_cifar10),])#测试集的转换

train_dataset = CIFAR10H(root='./data', train=False, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
print('train samples:',len(train_dataset), 'test samples:',len(test_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)
'''train_loader：使用 DataLoader 将训练数据集包装起来。设置 batch_size 为指定的批次大小，shuffle=True 表示在每个训练周期前随机打乱数据，以提高模型的泛化能力。num_workers=2 表示使用两个子进程来加载数据，加速数据读取。
test_loader：类似于训练集的数据加载器，但 shuffle=False，因为在测试时不需要打乱数据。'''

model = models.resnet34(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, args.num_classes)
model = model.to(device)
'''models.resnet34(pretrained=True)：加载预训练的 ResNet-34 模型。pretrained=True 表示使用在 ImageNet 数据集上训练的权重，这可以加速收敛并提高模型性能。
model.fc = nn.Linear(model.fc.in_features, args.num_classes)：替换模型的最后一层（全连接层），使其输出的类别数量与 CIFAR-10 数据集的类别数量相匹配（即 args.num_classes）。
model = model.to(device)：将模型移动到 GPU（如果可用）或 CPU。'''

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()
'''optim.SGD(...)：使用随机梯度下降（SGD）优化器。参数包括：
model.parameters()：优化器将优化模型的参数。
lr=args.lr：学习率。
momentum=0.9：动量项，帮助加速SGD在相关方向上的收敛。
nesterov=False：不使用 Nesterov 动量。
weight_decay=0.0001：L2 正则化，防止过拟合。'''


best_epoch, best_acc = 0.0, 0
for epoch in range(args.num_epoch):
    train(model, train_loader, criterion, optimizer)
    accuracy = test(model, test_loader)
    if accuracy > best_acc:
        patience = 0
        best_acc = accuracy
        best_epoch = epoch
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), 'best_model_cifar10h.pth.tar')
    print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
            epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

100%|██████████| 170M/170M [00:18<00:00, 9.23MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
train samples: 10000 test samples: 50000


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 156MB/s]


epoch: 0  acc: 0.3877  best epoch: 0  best acc: 0.3877
epoch: 1  acc: 0.5257  best epoch: 1  best acc: 0.5257
epoch: 2  acc: 0.6194  best epoch: 2  best acc: 0.6194
epoch: 3  acc: 0.6913  best epoch: 3  best acc: 0.6913
epoch: 4  acc: 0.1000  best epoch: 3  best acc: 0.6913
epoch: 5  acc: 0.1000  best epoch: 3  best acc: 0.6913
epoch: 6  acc: 0.2263  best epoch: 3  best acc: 0.6913
epoch: 7  acc: 0.4069  best epoch: 3  best acc: 0.6913
epoch: 8  acc: 0.4915  best epoch: 3  best acc: 0.6913
epoch: 9  acc: 0.4941  best epoch: 3  best acc: 0.6913
epoch: 10  acc: 0.5760  best epoch: 3  best acc: 0.6913
epoch: 11  acc: 0.5736  best epoch: 3  best acc: 0.6913
epoch: 12  acc: 0.5678  best epoch: 3  best acc: 0.6913
epoch: 13  acc: 0.5400  best epoch: 3  best acc: 0.6913
epoch: 14  acc: 0.6472  best epoch: 3  best acc: 0.6913
epoch: 15  acc: 0.6299  best epoch: 3  best acc: 0.6913
epoch: 16  acc: 0.6520  best epoch: 3  best acc: 0.6913
epoch: 17  acc: 0.6176  best epoch: 3  best acc: 0.6913
ep

In [ ]:
import tensorflow_probability as tfp

def evaluation_all(model, testloader):
    model.eval()
    logits_list = []
    labels_list = []
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1) #outputs returns the max value and index of the maximum output in the dimension 1
            total += targets.size(0) #size of 0th dimension-batch size
            correct += predicted.eq(targets).sum().item()#predicted.eq(targets).sum().item() 的最终目的是计算在当前批次中，模型预测正确的样本数量。.sum求一个布尔张量中true的个数，结果是一个张量，item帮助将张量转化为python数字
            logits_list.append(outputs)
            labels_list.append(targets)

        logits = torch.cat(logits_list).cpu().numpy()
        labels = torch.cat(labels_list).cpu().numpy()
    return correct / total, logits, labels

model.load_state_dict(torch.load('best_model_cifar10h.pth.tar'))#load the best combination of parameters to current model
acc, logits_tf, labels_tf = evaluation_all(model, test_loader)
ece = tfp.stats.expected_calibration_error(args.num_classes, logits=logits_tf, labels_true=labels_tf, labels_predicted=np.argmax(logits_tf,1))
print("Acc:{:.4f}, ECE:{:.4f}".format(acc, np.array(ece)))

<ipython-input-9-55214cfbd0b7>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_cifar10h.pth.tar'))


Acc:0.7400, ECE:0.1870
